# 크롤링

In [1]:
# 크롤러를 만들기 전 필요한 도구들을 임포트합니다.
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 페이지 수, 카테고리, 날짜를 입력값으로 받습니다.
def make_urllist(page_num, code1, code2, date): 
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm'+'&sid2='+str(code2)+'&sid1='+str(code1)+'&date='+str(date)+'&page='+str(i)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
    news = requests.get(url, headers=headers)

    # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

In [9]:
from datetime import datetime
datetime.today().strftime("%Y%m%d")

url_list = make_urllist(2, 105, 283, datetime.today().strftime("%Y%m%d"))
print('뉴스 기사의 개수: ',len(url_list))
url_list[:5]

뉴스 기사의 개수:  27


['https://n.news.naver.com/mnews/article/138/0002145590?sid=105',
 'https://n.news.naver.com/mnews/article/138/0002145589?sid=105',
 'https://n.news.naver.com/mnews/article/014/0004990825?sid=105',
 'https://n.news.naver.com/mnews/article/092/0002287443?sid=105',
 'https://n.news.naver.com/mnews/article/031/0000738317?sid=105']

In [10]:
idx2word = {'101' : '경제', '102' : '사회', '103' : '생활/문화', '105' : 'IT/과학'}
idx2word2 = {'731' : '모바일', '226' : '인터넷/SNS', '227' : '통신/뉴미디어', '230' : 'IT 일반', '732' : '보안/해킹', '283' : '컴퓨터'}

In [14]:
from newspaper import Article

#- 데이터프레임을 생성하는 함수입니다.
def make_data(urllist, code1, code2):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
  df = pd.DataFrame({'news': text_list})

  #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
  df['code1'] = idx2word[str(code1)]
  df['code2'] = idx2word2[str(code2)]
  return df

In [15]:
data = make_data(url_list, 105, 283)
#- 상위 10개만 출력해봅니다.
data[:10]

,news,code1,code2
0,\n\n\n\n웹케시그룹은 이번 공채를 통해 ▲SW개발 ▲UI/UX디자인 ▲상품 ▲...,IT/과학,컴퓨터
1,[디지털데일리 이종현기자] 작년은 클라우드 업계에서 기념비적인 해라고 할 만합니다....,IT/과학,컴퓨터
2,[파이낸셜뉴스] NHN클라우드가 독립법인 출범 1주년을 맞이했다. 지난해 성과에 힘...,IT/과학,컴퓨터
3,시놀로지가 영상 보안 시스템 정보 유출을 막을 수 있는 가이드라인을 공개했다. (사...,IT/과학,컴퓨터
4,인공지능(AI) 스타트업 업스테이지는 카카오톡 아숙업(AskUp)에 이미지 생성 A...,IT/과학,컴퓨터
5,\n\n\n\n시프티는 올해 창업 5년차 기업으로 동명의 인력관리 솔루션 ‘시프티’...,IT/과학,컴퓨터
6,삼성SDS는 국내 최초로 '소프트웨어(SW) 프로세스 품질인증(SP인증)' 최고등급...,IT/과학,컴퓨터
7,"웹 환경의 발전으로 대부분의 IT 서비스 시스템이 확장성, 유연성, 독립성을 제공하...",IT/과학,컴퓨터
8,토스플레이스는 애플페이 결제를 지원하기 위해 국제 결제 표준 규격인 EMV 컨택트리...,IT/과학,컴퓨터
9,\n\n\n\nKG스틸은 이번 프로젝트로 기존 ERP 시스템인 ‘SAP ECC 6....,IT/과학,컴퓨터


In [28]:
code_list = [[102,250], [103,243], [105,283]]
idx2word2 = {'731' : '모바일', '226' : '인터넷/SNS', '227' : '통신/뉴미디어', '230' : 'IT 일반', '732' : '보안/해킹', '283' : '컴퓨터', '250' : '교육', '243' : '책'}

code_list

[[102, 250], [103, 243], [105, 283]]

In [27]:
from multiprocessing import Pool
import random
import time, os

def make_total_data(page_num, code_list, date):
  start = int(time.time())  
  num_cores = 4  
  df = None
  for code1, code2 in code_list:
    pool = Pool(num_cores)
    url_list = make_urllist(page_num, code1, code2, date)
    df_temp = make_data(url_list, code1, code2)
    print(str(code1)+'번:'+str(code2)+'번'+' 코드에 대한 데이터를 만들었습니다.')
    pool.close()
    pool.join()
    time.sleep(random.randint(0,1))
    if df is not None:
      df = pd.concat([df, df_temp])
    else:
      df = df_temp

  print("***run time(sec) :", int(time.time()) - start)
  return df

In [29]:
df = make_total_data(1, code_list, datetime.today().strftime("%Y%m%d"))

102번:250번 코드에 대한 데이터를 만들었습니다.
103번:243번 코드에 대한 데이터를 만들었습니다.
105번:283번 코드에 대한 데이터를 만들었습니다.
***run time(sec) : 25


In [36]:
import os

# 데이터프레임 파일을 csv 파일로 저장합니다.
# 저장경로는 이번 프로젝트를 위해 만든 폴더로 지정해 주세요.
csv_path = os.path.join(os.getcwd(), '/news')
df.to_csv(csv_path+'/news.csv', index=False)

if os.path.exists(csv_path):
  print('{} File Saved!'.format(csv_path))

c:/news File Saved!


In [43]:
import pandas as pd
csv_path = os.path.join(os.getcwd(), '/news')
df = pd.read_table(csv_path+'/news.csv', sep=',')
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  58


In [45]:
from konlpy.tag import Mecab
tokenizer = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")
kor_text = '밤에 귀가하던 여성에게 범죄를 시도한 대 남성이 구속됐다서울 제주경찰서는 \
            상해 혐의로 씨를 구속해 수사하고 있다고 일 밝혔다씨는 지난달 일 피해 여성을 \
            인근 지하철 역에서부터 따라가 폭행을 시도하려다가 도망간 혐의를 받는다피해 \
            여성이 저항하자 놀란 씨는 도망갔으며 신고를 받고 주변을 수색하던 경찰에 \
            체포됐다피해 여성은 이 과정에서 경미한 부상을 입은 것으로 전해졌다'

#- 형태소 분석, 즉 토큰화(tokenization)를 합니다.
print(tokenizer.morphs(kor_text))

['밤', '에', '귀가', '하', '던', '여성', '에게', '범죄', '를', '시도', '한', '대', '남성', '이', '구속', '됐', '다', '서울', '제주', '경찰서', '는', '상해', '혐의', '로', '씨', '를', '구속', '해', '수사', '하', '고', '있', '다고', '일', '밝혔', '다', '씨', '는', '지난달', '일', '피해', '여성', '을', '인근', '지하철', '역', '에서부터', '따라가', '폭행', '을', '시도', '하', '려다가', '도망간', '혐의', '를', '받', '는다', '피해', '여성', '이', '저항', '하', '자', '놀란', '씨', '는', '도망갔으며', '신고', '를', '받', '고', '주변', '을', '수색', '하', '던', '경찰', '에', '체포', '됐', '다', '피해', '여성', '은', '이', '과정', '에서', '경미', '한', '부상', '을', '입', '은', '것', '으로', '전해졌', '다']


In [65]:
song = '''식사는 없어 배고파도
음료는 없어 목말라도
달콤한 맛만 디저트만 만 원하게 될 거 알잖아
식사는 없어 배고파도
음료는 없어 목말라도
달콤한 맛만 디저트만 만 원하게 될 거 알잖아
내가 만든 쿠키'''

In [67]:
tokenizer.morphs(song)

['식사',
 '는',
 '없',
 '어',
 '배고파도',
 '음료',
 '는',
 '없',
 '어',
 '목말',
 '라도',
 '달콤',
 '한',
 '맛',
 '만',
 '디저트',
 '만',
 '만',
 '원',
 '하',
 '게',
 '될',
 '거',
 '알',
 '잖아',
 '식사',
 '는',
 '없',
 '어',
 '배고파도',
 '음료',
 '는',
 '없',
 '어',
 '목말',
 '라도',
 '달콤',
 '한',
 '맛',
 '만',
 '디저트',
 '만',
 '만',
 '원',
 '하',
 '게',
 '될',
 '거',
 '알',
 '잖아',
 '내',
 '가',
 '만든',
 '쿠키']

In [37]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [48]:
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))
  return text_data


In [51]:
text_data = preprocessing(df['news'])
text_data

['기사 내용 요약 승마 체험 비 만 원 중 인 만 원 지원 거제 뉴시스 신정철 경남 거제시 말 산업 육성 승마 인구 저변 확대 를 위해 관내 초중 고등학교 재학 생 대상 으로 학생 승마 체험 지원 사업 추진 한다고 일 밝혔 학생 승마 체험 지원 사업 매년 월 부터 월 까지 실시 며 승마 체험 비 만 원 중 인 만 원 지원 한다 올해 억 만 원 자부담 포함 투입 해 명 대상 으로 지원 한다 희망자 중 대상자 로 확정 된 학생 동부면 소재 다솔 승마 장 에서 주말 토일 오전 시 부터 오후 시 까지 안전교육 승마 이론 교육 말타기 으로 이루어진 체험 학생 당 회회 당 분 걸쳐 진행 한다 거 제시 관계자 학생 승마 체험 통해 말 같 큰 동물 교감 함 으로써 정서 적 안정 바른 체형 교정 여 건강 몸 단련 데 도움 되 기 를 바란다고 말',
 '전북 임충식 전라북도 교육청 교육감 서거석 학생 적성 흥미 맞 맞춤 형 진로 진학 상담 제공 위해 권역 별 대면 진학 상담 운영 한다고 일 밝혔 대상 전라북도 소재 고등학교 학생 졸업 생 검정고시 생 학부모 다상담 오 일 부터 월 까지 요일 별 로 도 교육청 각 교육 지원 청 에서 이뤄진 시간 평일 오후 시분 부터 시분 까지 인 당 상담 시간 분 신청 시간 분 전 상담 장소 방문 등록 후 상담 진행 면 된 다상담 년 이상 진학 지도 경험 전북 대 입진 학지 도 지원 단 진학 상담 팀 교사 맡 게 된다 진학 상담 참여 고 싶 학생 학부모 전북 진로 진학 센터 홈 페이 접속 해 대입 대면 상담 신청 배 너 를 클릭 고 상담 원하 장소 와 시간 확인 후 접수증 출력 면 신청 마무리 된 다상담 신청 일 오후 시 부터 가능 며 상담 희망 일 주일 전 까지 만 신청 면 희망 일 상담 받 기타 자세 신청 방법 이나 운영 일 전북 진로 진학 센터 홈페이지 에서 확인 가능 유효선 학교 교육 과장 학교 에서 진로 진학 상담 이뤄지고 지만 더 많 정보 필요 학생 학부모 를 위해 권역 별 진학 상담 마련 다면서 우리 교육청 학생 학부모 진학 정보 

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [53]:
#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code1'], random_state = 0)

In [55]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 43
테스트용 뉴스 기사의 개수 :  15
훈련용 레이블의 개수 :  43
테스트용 레이블의 개수 :  15


In [56]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [59]:
def tfidf_vectorizer(data):
    data_counts = count_vect.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

In [60]:
new_sent = preprocessing(["민주당 일각에서 법사위의 체계·자구 심사 기능을 없애야 한다는 \
                           주장이 나오는 데 대해 “체계·자구 심사가 법안 지연의 수단으로 \
                          쓰이는 것은 바람직하지 않다”면서도 “국회를 통과하는 법안 중 위헌\
                          법률이 1년에 10건 넘게 나온다. 그런데 체계·자구 심사까지 없애면 매우 위험하다”고 반박했다."])
print(clf.predict(tfidf_vectorizer(new_sent)))

['IT/과학']


In [61]:
new_sent = preprocessing(["인도 로맨틱 코미디 영화 <까립까립 싱글>(2017)을 봤을 때 나는 두 눈을 의심했다. \
                          저 사람이 남자 주인공이라고? 노안에 가까운 이목구비와 기름때로 뭉친 파마머리와, \
                          대충 툭툭 던지는 말투 등 전혀 로맨틱하지 않은 외모였다. 반감이 일면서 \
                          ‘난 외모지상주의자가 아니다’라고 자부했던 나에 대해 회의가 들었다.\
                           티브이를 꺼버릴까? 다른 걸 볼까? 그런데, 이상하다. 왜 이렇게 매력 있지? 개구리와\
                            같이 툭 불거진 눈망울 안에는 어떤 인도 배우에게서도 느끼지 못한 \
                            부드러움과 선량함, 무엇보다 슬픔이 있었다. 2시간 뒤 영화가 끝나고 나는 완전히 이 배우에게 빠졌다"])
print(clf.predict(tfidf_vectorizer(new_sent)))

['생활/문화']


In [62]:
new_sent = preprocessing(["20분기 연속으로 적자에 시달리는 LG전자가 브랜드 이름부터 성능, 디자인까지 대대적인 변화를 \
                          적용한 LG 벨벳은 등장 전부터 온라인 커뮤니티를 뜨겁게 달궜다. 사용자들은 “디자인이 예쁘다”, \
                          “슬림하다”는 반응을 보이며 LG 벨벳에 대한 기대감을 드러냈다."])
print(clf.predict(tfidf_vectorizer(new_sent)))

['IT/과학']


In [63]:
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.44      1.00      0.62         4
          사회       0.75      0.75      0.75         4
       생활/문화       1.00      0.29      0.44         7

    accuracy                           0.60        15
   macro avg       0.73      0.68      0.60        15
weighted avg       0.79      0.60      0.57        15

